In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [19]:
with open("links.txt") as f:
    text = f.read()
text = text.replace('"', '')
all_schools = {k[0].strip(): k[1].strip() for k in (school.split('-') for school in text.split('\n'))}

In [21]:
all_schools

{'Boston College': 'https://bceagles.com/index.aspx',
 'Clemson University': 'https://clemsontigers.com/',
 'Duke University': 'https://goduke.com/index.aspx',
 'Florida State University': 'https://seminoles.com/',
 'Georgia Institute of Technology (Georgia Tech)': 'https://ramblinwreck.com/',
 'University of Louisville': 'https://gocards.com/',
 'University of Miami': 'https://miamihurricanes.com/',
 'University of North Carolina at Chapel Hill': 'https://goheels.com/',
 'North Carolina State University': 'https://gopack.com/',
 'University of Notre Dame': 'https://fightingirish.com/',
 'University of Pittsburgh': 'https://pittsburghpanthers.com/index.aspx',
 'Syracuse University': 'https://cuse.com/',
 'University of Virginia': 'https://virginiasports.com/',
 'Virginia Polytechnic Institute and State University (Virginia Tech)': 'https://hokiesports.com/index.aspx',
 'Wake Forest University': 'https://godeacs.com/'}

In [71]:
page = BeautifulSoup(requests.get("https://virginiasports.com/sports/baseball/roster/").content, 'html.parser')
sports = [a.get("href").split('/')[2] for a in page.find_all('a') if 'roster' in a.get('href') and 'https' not in a.get('href')]
sports

['baseball',
 'mbball',
 'xc',
 'football',
 'mgolf',
 'mlax',
 'msoc',
 'msquash',
 'swimming',
 'mten',
 'xctrack',
 'wrestling',
 'wbball',
 'xc',
 'fhockey',
 'wgolf',
 'wlax',
 'wrow',
 'wsoc',
 'softball',
 'wsquash',
 'swimming',
 'wten',
 'xctrack',
 'wvball']

In [77]:
# page = BeautifulSoup(requests.get("https://virginiasports.com/sports/xc/roster/season/2014-15/").content, 'html.parser')
def get_roster(url):
    roster = pd.read_html(url)[0]
    cols = ['Name', 'Hometown', 'High School']
    data = roster[cols].rename(columns = {'Class': 'Academic Year'})
    return data

In [78]:
def all_years(sport):
    df = pd.DataFrame()
    for year in range(2011, 2024):
        print(year)
        season = str(year) + '-' + str(year + 1)[-2:]
        link = f"https://virginiasports.com/sports/{sport}/roster/season/{season}/"
        try:
            yearly = pd.DataFrame(get_roster(link))
            yearly['Year'] = year
            df = pd.concat([df, yearly])
        except Exception as error:
            print(link, error)
    df['sport'] = sport
    return df

In [70]:
pd.read_html("https://virginiasports.com/sports/xc/roster/season/2014-15/")

[           Name  Position
 0  Bryan Fetzer       NaN]

In [79]:
all_players = pd.DataFrame()
# Update Sports for school
for sport in set(sports):
    print(sport)
    all_players = pd.concat([all_players, all_years(sport)])

msquash
2011
https://virginiasports.com/sports/msquash/roster/season/2011-12/ No tables found
2012
https://virginiasports.com/sports/msquash/roster/season/2012-13/ No tables found
2013
https://virginiasports.com/sports/msquash/roster/season/2013-14/ No tables found
2014
https://virginiasports.com/sports/msquash/roster/season/2014-15/ No tables found
2015
https://virginiasports.com/sports/msquash/roster/season/2015-16/ No tables found
2016
https://virginiasports.com/sports/msquash/roster/season/2016-17/ No tables found
2017
2018
2019
2020
2021
2022
2023
wsquash
2011
https://virginiasports.com/sports/wsquash/roster/season/2011-12/ No tables found
2012
https://virginiasports.com/sports/wsquash/roster/season/2012-13/ No tables found
2013
https://virginiasports.com/sports/wsquash/roster/season/2013-14/ No tables found
2014
https://virginiasports.com/sports/wsquash/roster/season/2014-15/ No tables found
2015
https://virginiasports.com/sports/wsquash/roster/season/2015-16/ No tables found
201

In [86]:
all_players.to_csv("done/virginia.csv")

In [61]:
import copy
test = copy.deepcopy(all_players)
substrings_to_keep = [' Rs.', ' Sr.', ' Jr.', ' So.', ' Fr.', ' R-Fr.', ' R-So.', ' R-Jr.', ' Gr.', ' R-Sr.', ' 5th']
# all_players[all_players['Position'].str.contains('|'.join(substrings_to_keep))]
# test.loc[test['Position'].isin(substrings_to_keep), 'C']

In [62]:
test.loc[test['Position'].isin(substrings_to_keep), "Weight"] = test.loc[test['Position'].isin(substrings_to_keep), "Height"]

In [63]:
test.loc[test['Position'].isin(substrings_to_keep), "Height"] = test.loc[test['Position'].isin(substrings_to_keep), "Academic Year"]
test.loc[test['Position'].isin(substrings_to_keep), "Academic Year"] = test.loc[test['Position'].isin(substrings_to_keep), "Position"]

In [64]:
test.loc[test['Position'].isin(substrings_to_keep), "Position"] = "NA"

In [66]:
test.loc[test['Academic Year'] == " 6' 5'' ", 'Height'] = test.loc[test['Academic Year'] == " 6' 5'' ", "Academic Year"]
test.loc[test['Academic Year'] == " 6' 5'' ", 'Academic Year'] = "NA"

In [72]:
test.loc[test['Height'].str.contains('lbs'), 'Weight'] = test.loc[test['Height'].str.contains('lbs'), 'Height']
test.loc[test['Height'].str.contains('lbs'), 'Height'] = "NA"

In [99]:
test.to_csv(f"All_{school}.csv")